In [ ]:
from azureml.core import Workspace, Dataset, Experiment, Run
from azureml.core.compute import ComputeTarget,AmlCompute
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute_target import ComputeTargetException
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import logging, requests,json
import urllib.request

# First things first: the Workspace. You use the Workspace to experiment, train and deploy machine learning models.
# You can create a brand new one from zero or you can get the configuration of the one you're using.
# Are you able to find out how to gather the configuration of your current workspace?
# 
# TIPS: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py#

ws = Workspace.#find the right method to gather the current workspace info

# In this demo we already uploaded the dataset.
# How can you use it and reference it in the code? 
# TIPS: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py
dataset = Dataset.#reference the dataset you uploaded before

# Since all the imported dataset are formatted as Tabular,let's 
# transform the dataset into a pandas dataframe for easier use.
df = dataset.to_pandas_dataframe()

In [ ]:
# In every ML project, the first step we should do is to pre-process the data.
# The dataset we are using is already pre-processed so we can skip this step
# and get to the next one: splitting the dataset. 

# We'll split the dataset in train and test
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
# We define the label variable, we'll need it for AutoML
label = "quality"


In [ ]:
# AutoML is a great tool! Allows us to find quickly the best algorithm for our data.
# It needs to be configured and we'll need a few things:
# 1. Tabular train dataset
# 2. Compute target that will be use to run our experiment
# 3. An experiment to keep track of our training runs
# 4. Configure the AutoML run

# We split our dataset in train and test, but our dataset is of type 
# dataframe and not tabular. We need to convert it back to tabular to use it in our
# AutoML config run. How can we do that? 
# First, we need to access the default datastore connected to the Workspace.
# TIPS: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py

datastore = #How can you get the default datastore connected to your current workspace?

# Now that we have the datastore, we just need to register our dataframe into a Tabular dataset.
# We can give it a name and a description, after we register it we will be able to see it in our
# Datasets tab. 
#TIPS: https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/machine-learning/how-to-create-register-datasets.md#create-a-dataset-from-pandas-dataframe
train_dataset = Dataset.#How can you create and register a dataframe into a Tabular dataset? 


In [ ]:
# We need now to define the compute instance we are going to use to run AutoML
# To do so, we first need to give it a name. If you already created a compute resource
# and want to use it, you can use that same name. 
# This code will check if the compute exists, if not it will create it 

compute_name = 'edit_here_your_compute_name' #Edit with your compute name

try:
    compute_target = ComputeTarget(workspace=ws,name=compute_name)
    print('Found existing compute resource, using it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size= 'STANDARD_D12_V2', #Edit the vm_size if you prefer a bigger or smaller vm
        max_nodes=2 #Edit number of nodes if you want to give different configurations
    )
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [ ]:
# Now we need to configure the AutoML run 
# To do so, we need to define:
# - what kind of task we are trying to do
# - which metric we will use to define how is our model performing 
# - experiment timeout (for the sake of the demo)
# - the training tabular data
# - the label
# - the compute targe
# - cross-validation number
# TIPS: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
# Automl config
automl_config = AutoMLConfig(
                                 task='type_of_task', # is it classification, regression or forecasting?
                                 debug_log ='automl_notebook_errors.log',
                                 primary_metric='AUC_weighted',
                                 experiment_timeout_minutes=30,
                                 training_data=#which is your training dataset?,
                                 label_column_name=#which is the label?,
                                 compute_target = #which is the target compute?,
                                 n_cross_validations=5,
                                 verbosity=logging.INFO,
                                 featurization='auto',
                                 enable_voting_ensemble=False,
                                 enable_stack_ensemble=False
                                )

In [ ]:
# It's time to set up our Experiment, which will contain all the runs and info on the training.
# Give it a name that is significant for your training 

experiment_name = 'edit_experiment_name'#Edit the experiment name

# Let's create the experiment in the current workspace. 
# TIPS: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py
experiment = Experiment #Create the experiment with the name you decided in the current workspace

#Submit the experiment and get the created run
run = experiment.# how can you submit the experiment using AutoML?


In [ ]:
# After it finished running your data on the different algorithms
# AutoML gives you the best run, the one with the best value on the metric
# you defined in the configuration. 
# Can you retrieve the best run and the best model? 
# TIPS: https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.run.automlrun?view=azure-ml-py
best_run, fitted_model = #How can you retrieve the best run and the model? 
print(fitted_model.steps)

In [ ]:
# Before the deployment of the model, let's test it on our test data.
# To check the performance

# Drop the label column from the test dataframe
y_test = test_data.pop("quality")

# Let's call the predict model method on the data and store predictions
y_classify = fitted_model.predict(test_data)

# Let's easily print the first 10 prediction and actual value to compare
# You could even do some better analysis on this results
print("Predicted:  " + y_classify[:10] + " Value: " + y_test[:10])


In [ ]:
# We are almost done with our ML process! 
# We found the best model fitting our data and now we are close to deploying it and consume it!
# As we said previously, we need first to register the model in the current workspace. 
# To keep track of the model and publish always the best version of it.
# How can we do this step? 
# We get the model name, we add a description and we register it.
#TIPS: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train

#Model info
model_name = #From the run, retrieve the model name
description = 'AutoML jupyter example'
tags = None

#Register the model
#TIPS: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py
model = run.#register the model

In [ ]:
# Last step to deploy our registered model is configure the web service we are going to consume.
# Since it's going to be an api we need to define the script that will run when the API gets called
# We can manually write the script or use an auto-generated script AutoML gives us. 
# We are going to retrieve and use the script.
# TIPS: 

# Let's give a name to the script
script_file_name = 'score.py'
# The script we want to retrieve is at this path:
# ./outputs/scoring_file_v_1_0_0.py
# How would you retrieve it? 
#TIPS: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py

file_dwn = best_run.#retrieve the auto generated scoring file 


In [ ]:
# Finally the last step: deploy the model into a container
# We have our script, we have the model registered. 
# Now let's consume and use the model. 
# To do so, we need to configurations:
# 1. our environment and entry script
# 2. the container instance we are going to use

# Let's start from the environment configuration. 
# The model is ready and registered and the run has all the information we need. 
# TIPS: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py
inference_config = InferenceConfig(environment = best_run.#get the environment info, 
                                   entry_script = './score.py') #we saved the script in the root folder, let's reference it

#You can change the following configuration fo the container instance
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 4, 
                                               tags = {'type': "automl-jupyter"},
                                               description = "Automl jupyter sample service")

aci_service_name = 'automl-jupyter-model'
print(aci_service_name)
# It's time to deploy our model! How can you do it?
#TIPS: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py
aci_service = #deploy the domdel
aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
# And now, let's consume and test our model with new data!
# We need the scoring uri to call the service
# TIPS: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.aci.aciwebservice?view=azure-ml-py
scoring_uri = # how can you retrieve the scoring uri?

# Let's try sending this data
data = {"data": [
    {
      "fixed acidity": 8.8,
      "volatile acidity": 0.32,
      "citric acid": 0.57,
      "residual sugar": 2.3,
      "chlorides": 0.069,
      "free sulfur dioxide": 29,
      "total sulfur dioxide": 37,
      "density": 0.997,
      "pH": 3.15,
      "sulphates": 0.61,
      "alcohol": 9.6
    }
  ],
  "method": "predict"
  }

input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print(resp.status_code)
print("The wine submitted is " + resp.json())